# Challenges 1>* solutions

From the hackathon website https://wirvsvirushackathon.devpost.com/

In [333]:
import requests
import json
import pandas as pd
import altair as alt
from tqdm import tqdm
from bs4 import BeautifulSoup

In [334]:
BASEURL = 'https://wirvsvirushackathon.devpost.com/'

# Get the project gallery

In [335]:
gallery = requests.get(f"{BASEURL}project-gallery")
gallery_soup = BeautifulSoup(gallery.text, 'html.parser')

# Get the category pages

In [64]:
[(herausforderung['name'], herausforderung['value']) for herausforderung
 in gallery_soup.select('.filter-submissions > .panel > ul > li > label > input')]

[('filter[bitte wählt eure herausforderung aus:][]',
  '1 - lebensmittel-matching'),
 ('filter[bitte wählt eure herausforderung aus:][]', '2- tauschplattform'),
 ('filter[bitte wählt eure herausforderung aus:][]',
  '3- arbeiterinnenverteilung'),
 ('filter[bitte wählt eure herausforderung aus:][]',
  '4 - verteilung von helferinnen'),
 ('filter[bitte wählt eure herausforderung aus:][]',
  '5 - hilfsmittelverteilung'),
 ('filter[bitte wählt eure herausforderung aus:][]',
  '6 - medizingeräteherstellung'),
 ('filter[bitte wählt eure herausforderung aus:][]', '7 - e-kinderbetreuung'),
 ('filter[bitte wählt eure herausforderung aus:][]', '8 - corona tracking'),
 ('filter[bitte wählt eure herausforderung aus:][]', '9 - e-anträge'),
 ('filter[bitte wählt eure herausforderung aus:][]',
  '10 - analoge unterstützung'),
 ('filter[bitte wählt eure herausforderung aus:][]',
  '11 - infektionsfall-Übermittlung'),
 ('filter[bitte wählt eure herausforderung aus:][]', '12 - social distancing'),
 ('fi

In [161]:
herausforderung = [requests.get(f"{BASEURL}submissions/search", {k: v}) for k, v
                   in [(herausforderung['name'], herausforderung['value']) for herausforderung
                       in gallery_soup.select('.filter-submissions > .panel:nth-of-type(1) > ul > li > label > input')]]

A bit of testing for to see we got what we expected

In [336]:
assert len(herausforderung) == 48
assert all([h.ok for h in herausforderung])

In [337]:
herausforderung_soups = [BeautifulSoup(h.text, 'html.parser') for h in herausforderung]

# How many entries per herausforderung?

In [143]:
[int(h.select('.items_info > p > b:nth-of-type(2)')[0].text) for h in herausforderung_soups]

[62,
 8,
 28,
 14,
 23,
 25,
 24,
 96,
 19,
 33,
 23,
 98,
 6,
 13,
 13,
 103,
 40,
 75,
 76,
 35,
 27,
 9,
 51,
 53,
 37,
 27,
 5,
 24,
 12,
 15,
 37,
 14,
 18,
 45,
 25,
 7,
 10,
 71,
 23,
 24,
 9,
 4,
 23,
 19,
 6,
 56,
 24,
 9,
 1039,
 196,
 49]

Align the titles with counts

In [338]:
solution_counts = pd.DataFrame(
    zip([h['value'] for h in gallery_soup.select('.filter-submissions > .panel > ul > li > label > input')],
        [int(h.select('.items_info > p > b:nth-of-type(2)')[0].text) for h in herausforderung_soups]),
    columns=['name', 'count'])

In [339]:
solution_counts.head()

,name,count
0,1 - lebensmittel-matching,62
1,2- tauschplattform,8
2,3- arbeiterinnenverteilung,28
3,4 - verteilung von helferinnen,14
4,5 - hilfsmittelverteilung,23


In [340]:
solution_counts.index = [s[0].strip() for s in solution_counts.name.str.split('-')]

In [341]:
solution_counts.name = [s[1].strip() for s in solution_counts.name.str.split('-', 1)]

In [342]:
solution_counts.head()

,name,count
1,lebensmittel-matching,62
2,tauschplattform,8
3,arbeiterinnenverteilung,28
4,verteilung von helferinnen,14
5,hilfsmittelverteilung,23


In [343]:
alt.Chart(solution_counts.reset_index()).mark_bar().encode(
    x='name:O',
    y='count',
    tooltip=['name', 'count']
)

alt.Chart(...)

# Solution counts

In [344]:
challenge_counts = pd.read_csv('categories_count_with_ids.csv', sep=';', index_col='id')

In [345]:
challenge_counts.index = solution_counts.index.astype(str, copy = False)

In [346]:
challenge_counts.sort_index().head()

,name,count,category
1,Analoge Unterstützung,23,analog
10,e-Learning,42,education
11,e-Sport,8,fun
12,Finanzelle Unterstützung,1,economy
13,Flatten the Curve,1,NaN


In [347]:
df = pd.concat([
        solution_counts.drop(['name'], axis=1).rename({'count': 's_count'}, axis=1),
        challenge_counts.rename({'count': 'c_count'}, axis=1)],
    axis=1)

In [348]:
df.head()

,s_count,name,c_count,category
1,62,Analoge Unterstützung,23,analog
2,8,ArbeiterInnenverteilung,46,matching and distribution
3,28,Corona Tracking,58,tracking
4,14,Corona-Testprozesse,22,information exchange
5,23,Daten,17,data


In [349]:
df.to_csv('challenges_and_solution_counts.csv')

In [351]:
chart = alt.Chart(df.reset_index())

chart.mark_line().encode(x='name', y='c_count') + chart.mark_line(color='hotpink').encode(x='name', y='s_count')

alt.LayerChart(...)